# Let's train a classifier

In [1]:
import os
# os.system("pip install pandas")
# os.system("pip install torchvision")
os.system("CUDA_LAUNCH_BLOCKING=1")
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.models import resnet34, ResNet34_Weights

import time

import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

torch.manual_seed(1)

In [2]:
# choose experiment
size_train = 10000 #86744
experiments = ["FairFace", "CelebA", "CelebA only white", "CelebA augmented"]
exp = 1

# set celeb paths
celeb_attr_path = "datasets/celeba/list_attr_celeba.txt"
celeb_partitions_path = 'datasets/celeba/list_eval_partition.txt'
celeb_race_path = "CelebA/races/races_ff.csv"
celeb_label_dir = "CelebA/labels_split/"
celeb_img_dir = "CelebA/cropped/"
celeb_img_aug_dir = "CelebA/augmented/"
celeb_train_csv = f"train_{size_train}_samples_random.csv" # "train_total.csv"
celeb_train_only_white_csv = f"train_{size_train}_samples_random_white.csv"
celeb_train_aug_csv = f"train_aug_{size_train}_samples.csv"
celeb_val_csv = "val_total.csv"
celeb_test_csv = "test_total.csv"


# set fairface paths
ff_img_dir = "fairface/dataset/fairface-img-margin125-trainval"
ff_label_dir = "fairface/dataset/"
ff_train_csv = "fairface_label_train.csv"
ff_val_csv = "fairface_label_val.csv"


# set hyperparameters
learning_rates = [5e-5, 5e-5, 5e-5, 2e-5]
lr = learning_rates[exp]
num_epochs = 10

# Architecture
feat_size = (256, 256)
bs_train = 128
bs_val = 128
bs_test = 128
device = 'cuda:3'


races = ["Black", "Indian", "Latino", "Middle Eastern", "Southeast Asian", "East Asian", "White"]
ignored_attributes = ["Black_Hair", "Blond_Hair", "Brown_Hair", "Pale_Skin"]

In [3]:
# define datasets
class CelebaDataset(Dataset):
    """Custom Dataset for loading CelebA face images"""

    def __init__(self, csv_path, img_dir, transform=None, ignored_attributes=[]):
    
        df = pd.read_csv(csv_path, index_col=None)
        # print(df.head())
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names = df["Image_Name"].values
        self.races = df["Race"].values
        drop_cols = ["Image_Name", "Race"] + ignored_attributes
        self.y = np.expand_dims(np.array(df["Male"].values), axis=1) #df.drop(drop_cols, axis=1).values #
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_names[index]))
        
        if self.transform is not None:
            img = self.transform(img)
        
        label = self.y[index]
        gt_race = self.races[index]
        return img, label, gt_race

    def __len__(self):
        return self.y.shape[0]
    


class FairFaceDataset(Dataset):
    """Custom Dataset for loading FairFace images"""

    def __init__(self, csv_path, img_dir, transform=None):
    
        df = pd.read_csv(csv_path, index_col=None)
        # print(df.head())
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names = df["file"].values
        self.races = df["race"].replace("Latino_Hispanic", "Latino").values
        gender = df["gender"].replace("Male", 1).replace("Female", 0)
        self.y = np.expand_dims(np.array(gender.values), axis=1)
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_names[index]))
        
        if self.transform is not None:
            img = self.transform(img)
        
        label = self.y[index]
        gt_race = self.races[index]
        return img, label, gt_race

    def __len__(self):
        return self.y.shape[0]

In [4]:
# create datasets based on current experiment
num_workers = 6
custom_transform = transforms.Compose([transforms.Resize(feat_size),
                                       transforms.ToTensor()])

# training dataset
if experiments[exp].startswith("CelebA"):

    if experiments[exp].endswith("augmented"):
        train_csv = celeb_train_aug_csv
        train_img_dir = celeb_img_aug_dir
    
    else:
        if "only white" in experiments[exp]:
            train_csv = celeb_train_only_white_csv
        else:
            train_csv = celeb_train_csv
        train_img_dir = celeb_img_dir

    train_dataset = CelebaDataset(csv_path=celeb_label_dir + train_csv,
                                img_dir=train_img_dir,
                                transform=custom_transform,
                                ignored_attributes=ignored_attributes)

if experiments[exp].startswith("FairFace"):
    train_dataset = FairFaceDataset(csv_path=ff_label_dir + ff_train_csv,
                                    img_dir=ff_img_dir,
                                    transform=custom_transform)


# validation dataset
val_dataset = FairFaceDataset(csv_path=ff_label_dir + ff_val_csv,
                                img_dir=ff_img_dir,
                                transform=custom_transform)

# val_dataset = CelebaDataset(csv_path=celeb_label_dir + celeb_val_csv,
#                             img_dir=celeb_img_dir,
#                             transform=custom_transform,
#                             ignored_attributes=ignored_attributes)


# test datasets
test_dataset_celeb = CelebaDataset(csv_path=celeb_label_dir + celeb_test_csv,
                            img_dir=celeb_img_dir,
                            transform=custom_transform,
                            ignored_attributes=ignored_attributes)

test_dataset_ff = val_dataset


# create dataloaders on these datasets
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=bs_train,
                          shuffle=True,
                          num_workers=num_workers)

val_loader = DataLoader(dataset=val_dataset,
                          batch_size=bs_val,
                          shuffle=False,
                          num_workers=num_workers)

test_loader_celeb = DataLoader(dataset=test_dataset_celeb,
                          batch_size=bs_test,
                          shuffle=False,
                          num_workers=num_workers)

test_loader_ff = DataLoader(dataset=test_dataset_ff,
                          batch_size=bs_test,
                          shuffle=False,
                          num_workers=num_workers)

In [5]:
# build model, define loss and create optimizer
model = resnet34(weights=ResNet34_Weights.DEFAULT)
model.to(device)
num_attr_predicted = train_dataset.y.shape[1]
fc_layer = nn.Linear(1000, num_attr_predicted, device=device)
sigmoid = nn.Sigmoid()
bin_ce = nn.BCELoss()
params = list(model.parameters()) + list(fc_layer.parameters())
optimizer = torch.optim.Adam(params, lr=lr)

In [6]:
# define evaluation procedure
def evaluate_metrics(model, data_loader, device, show_tqdm=False):

    correct_predictions = np.zeros(len(races))
    true_pos = np.zeros(len(races))
    true_neg = np.zeros(len(races))
    positive_preds = np.zeros(len(races))
    positive_targets = np.zeros(len(races))
    num_examples = np.zeros(len(races))
    total_examples = len(data_loader.dataset) 

    # total_it = int(np.ceil(total_examples / data_loader.batch_size))
    for _, (features, targets, gt_races) in tqdm(enumerate(data_loader), total=len(data_loader), desc="Evaluating", disable=not show_tqdm):

        features = features.to(device)
        probas = sigmoid(fc_layer(model(features)))
        prediction = (probas >= 0.5).cpu().numpy()
        targets = targets.numpy()

        # prepape annotated races for metric split afterwards
        gt_races = np.array([races.index(race) for race in gt_races])
        gt_races = np.expand_dims(gt_races, axis=1)
        gt_races = np.broadcast_to(gt_races, prediction.shape)

        # collect the necessary data split by annotated race
        for j in range(len(races)):
            correct_preds = (gt_races == j) & (prediction == targets)
            true_pos[j] += (correct_preds & (prediction == 1)).sum()
            true_neg[j] += (correct_preds & (prediction == 0)).sum()
            correct_predictions[j] += correct_preds.sum()
            positive_targets[j] += ((gt_races == j) & (targets == 1)).sum()
            positive_preds[j] += np.where(gt_races == j, prediction, 0).sum()
            num_examples[j] += (gt_races == j).sum()

    # calculate and return metrics    
    zero = 1e-10
    print("Race distribution:", num_examples/targets.shape[1], "Total:", total_examples)

    total_accuracy = correct_predictions.sum() / num_examples.sum()
    accuracies = correct_predictions / (num_examples + zero)
    accs_out = [f"{a:.2%}" for a in accuracies]
    max_acc_disparity = np.log(max(accuracies)/min(accuracies))

    total_precision = true_pos.sum() / (positive_preds.sum() + zero)
    precisions = [f"{p:.2%}" for p in true_pos / (positive_preds + zero)]

    total_recall = true_pos.sum() / (positive_targets.sum() + zero)
    recalls = [f"{r:.2%}" for r in true_pos / (positive_targets + zero)]
    return total_accuracy, accs_out, max_acc_disparity, total_precision, precisions, total_recall, recalls


def get_elapsed_time(start_time):
    elapsed = int(time.time() - start_time)
    m, s = divmod(elapsed, 60)
    h, m = divmod(m, 60)
    return f"{h}:{m:02d}:{s:02d}"

In [7]:
# Training loop
start_time = time.time()

print(f"Initiating experiment '{experiments[exp]}' with a lr of {lr} and {size_train} samples on device {device}")


for epoch in range(num_epochs):
    
    model.train()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {(epoch+1):02d}/{num_epochs:02d}")
    for _, (features, targets, _) in pbar:
        
        features = features.to(device)
        targets = targets.float().to(device)
            
        # forward and backward pass
        model_output = model(features)
        logits = sigmoid(fc_layer(model_output))
        loss = bin_ce(logits, targets)
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        optimizer.zero_grad()
        
        loss.backward()
        
        # update model params 
        optimizer.step()
        
        # if batch_idx == 0:
        #     break 

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        acc_total, accs, max_acc_disp, prec_total, precs, rec_total, recs = evaluate_metrics(model, val_loader, device)
        print(f"Evaluation epoch {(epoch+1):02d}/{num_epochs:02d}:")
        print(f"Total accuracy: {acc_total:.2%}\t| Accuracies:\t{accs} | Max disparity: {max_acc_disp:.4f}")
        print(f"Total precision: {prec_total:.2%}\t| Precisions:\t{precs}")
        print(f"Total recall: {rec_total:.2%}\t| Recalls:\t{recs}\n")
    
print(f"Total Training Time: {get_elapsed_time(start_time)}")

Initiating experiment 'CelebA' with a lr of 5e-05 and 86744 samples on device cuda:3


Epoch 01/10: 100%|██████████| 678/678 [03:54<00:00,  2.90it/s, loss=0.0995]


Race distribution: [1556. 1516. 1623. 1209. 1415. 1550. 2085.] Total: 10954
Evaluation epoch 01/10:
Total accuracy: 82.91%	| Accuracies:	['75.84%', '81.13%', '85.27%', '88.42%', '82.26%', '80.00%', '87.05%'] | Max disparity: 0.1535
Total precision: 81.48%	| Precisions:	['70.80%', '76.03%', '81.69%', '90.49%', '83.70%', '81.77%', '86.92%']
Total recall: 87.59%	| Recalls:	['90.11%', '90.57%', '90.04%', '92.50%', '81.77%', '77.35%', '89.39%']



Epoch 02/10: 100%|██████████| 678/678 [03:53<00:00,  2.90it/s, loss=0.0092]


Race distribution: [1556. 1516. 1623. 1209. 1415. 1550. 2085.] Total: 10954
Evaluation epoch 02/10:
Total accuracy: 81.10%	| Accuracies:	['74.61%', '78.36%', '82.69%', '86.10%', '80.28%', '81.29%', '84.22%'] | Max disparity: 0.1432
Total precision: 77.37%	| Precisions:	['69.31%', '71.84%', '76.61%', '86.52%', '78.01%', '79.02%', '80.90%']
Total recall: 90.83%	| Recalls:	['90.74%', '92.83%', '92.94%', '93.97%', '86.39%', '85.33%', '92.51%']



...

Race distribution: [1556. 1516. 1623. 1209. 1415. 1550. 2085.] Total: 10954
Evaluation epoch 08/10:
Total accuracy: 83.76%	| Accuracies:	['77.70%', '83.11%', '85.71%', '87.92%', '81.77%', '83.42%', '86.43%'] | Max disparity: 0.1236
Total precision: 81.98%	| Precisions:	['73.06%', '78.66%', '82.06%', '89.47%', '82.45%', '84.21%', '84.58%']
Total recall: 88.81%	| Recalls:	['89.61%', '90.57%', '90.54%', '92.99%', '82.45%', '82.37%', '91.44%']



Epoch 09/10: 100%|██████████| 678/678 [03:51<00:00,  2.93it/s, loss=0.0103]


Race distribution: [1556. 1516. 1623. 1209. 1415. 1550. 2085.] Total: 10954
Evaluation epoch 09/10:
Total accuracy: 83.77%	| Accuracies:	['78.02%', '83.91%', '86.44%', '86.85%', '81.84%', '82.39%', '86.43%'] | Max disparity: 0.1072
Total precision: 84.63%	| Precisions:	['77.50%', '81.30%', '84.31%', '90.07%', '85.25%', '85.90%', '87.42%']
Total recall: 84.69%	| Recalls:	['80.60%', '87.78%', '88.78%', '90.41%', '78.64%', '77.61%', '87.34%']



Epoch 10/10: 100%|██████████| 678/678 [03:54<00:00,  2.89it/s, loss=0.0025]


Race distribution: [1556. 1516. 1623. 1209. 1415. 1550. 2085.] Total: 10954
Evaluation epoch 10/10:
Total accuracy: 83.39%	| Accuracies:	['75.96%', '82.19%', '86.14%', '88.25%', '81.48%', '82.52%', '86.76%'] | Max disparity: 0.1500
Total precision: 82.36%	| Precisions:	['72.63%', '76.98%', '82.64%', '90.18%', '83.83%', '85.14%', '85.85%']
Total recall: 87.28%	| Recalls:	['85.36%', '91.50%', '90.67%', '92.62%', '79.73%', '78.89%', '90.29%']

Total Training Time: 0:43:18


In [8]:
# evaluate experiment on test sets
with torch.set_grad_enabled(False): # save memory during inference
    print(f"\nEvaluation of experiment: '{experiments[exp]}'\n")

    # evaluation CelebA
    acc_total, accs, max_acc_disp, prec_total, precs, rec_total, recs = evaluate_metrics(model, test_loader_celeb, device, show_tqdm=True)
    print("\nEvaluation CelebA test set:")
    print(f"Total accuracy: {acc_total:.2%}\t| Accuracies:\t{accs}")
    print(f"Maximum accuracy disparity: {max_acc_disp:.4f}")
    print(f"Total precision: {prec_total:.2%}\t| Precisions:\t{precs}")
    print(f"Total recall: {rec_total:.2%}\t| Recalls:\t{recs}\n")

    # evaluation FairFace
    acc_total, accs, max_acc_disp, prec_total, precs, rec_total, recs = evaluate_metrics(model, test_loader_ff, device, show_tqdm=True)
    print("\nEvaluation FairFace test set:")
    print(f"Total accuracy: {acc_total:.2%}\t| Accuracies:\t{accs}")
    print(f"Maximum accuracy disparity: {max_acc_disp:.4f}")
    print(f"Total precision: {prec_total:.2%}\t| Precisions:\t{precs}")
    print(f"Total recall: {rec_total:.2%}\t| Recalls:\t{recs}\n")



Evaluation of experiment: 'CelebA'



Evaluating: 100%|██████████| 156/156 [00:30<00:00,  5.17it/s]


Race distribution: [ 1461.   553.  1269.  1538.   311.  1777. 13053.] Total: 19962

Evaluation CelebA test set:
Total accuracy: 97.94%	| Accuracies:	['95.96%', '99.46%', '98.82%', '98.11%', '95.18%', '97.58%', '98.11%']
Maximum accuracy disparity: 0.0440
Total precision: 97.65%	| Precisions:	['96.50%', '98.58%', '97.97%', '98.00%', '95.10%', '98.01%', '97.77%']
Total recall: 97.01%	| Recalls:	['96.39%', '100.00%', '97.69%', '98.13%', '94.44%', '94.43%', '97.14%']



Evaluating: 100%|██████████| 86/86 [00:15<00:00,  5.63it/s]

Race distribution: [1556. 1516. 1623. 1209. 1415. 1550. 2085.] Total: 10954

Evaluation FairFace test set:
Total accuracy: 83.39%	| Accuracies:	['75.96%', '82.19%', '86.14%', '88.25%', '81.48%', '82.52%', '86.76%']
Maximum accuracy disparity: 0.1500
Total precision: 82.36%	| Precisions:	['72.63%', '76.98%', '82.64%', '90.18%', '83.83%', '85.14%', '85.85%']
Total recall: 87.28%	| Recalls:	['85.36%', '91.50%', '90.67%', '92.62%', '79.73%', '78.89%', '90.29%']

